# HighDimBiff Demo

You can run this 
* either on the cluster, by first converting this notebook to a script via ''jupyter nbconvert --to script run_mc.ipynb --template=nbconv.tpl'', then calling it from the command line via a slurm script with an extra argument specifying the amount of processes that can be allocated 
    * the conversion works on all notebook cells that have a tag 'export' (view -> cell toolbar -> tags)
* interactivly on your own computer with this notebook!


In [ ]:
if isdefined(:cluster)
    using ClusterManagers
    N_tasks = parse(Int, ARGS[1])
    N_worker = N_tasks 
    addprocs(SlurmManager(N_worker))
    @everywhere include("/p/tmp/maxgelbr/code/HighBifLib.jl/HighBifLib.jl")
else
    #addprocs(1)
    @everywhere include("HighBifLib.jl")
    import Plots
end

@everywhere using LSODA
@everywhere using LightGraphs
using JLD2, FileIO, Clustering
@everywhere using DifferentialEquations
@everywhere using Distributions
@everywhere using HighBifLib  

# these imports invoke a lot of warnings when executed with multiple processes
# this seems to me to be more a bug of julia than an actual problem with this code
# imported on a single process there are no warnings

# Example 1: Order Parameter of Kuramoto Osc. 

We vary the initial conditions and the coupling strength. This is just to demonstrate how these kind of experiments are setup

**To-Do/Comment:** I am still very confused why and in which cases the @everywhere is needed. In this case the code doesn't work without any of the everywheres, but this goes against most of the explanations in the documentation.

In [ ]:
@everywhere N = 4
@everywhere K = 0.5
@everywhere nd = Normal(0.5, 0.5) # distribution for eigenfrequencies # mean = 0.5Hz, std = 0.5Hz
@everywhere w_i_par = rand(nd,N) 

@everywhere net = erdos_renyi(N, 0.2)
@everywhere A = adjacency_matrix(net)

@everywhere ic = zeros(N)
@everywhere ic_ranges = [-pi:pi/2:pi/2 for i=1:N]
@everywhere K_range = 0.0:2:10.
@everywhere pars = kuramoto_network_parameters(K, w_i_par, N, A)

# base problem
@everywhere rp = ODEProblem(kuramoto_network, ic, (0.,100.), pars)

# setup the MC problem with the helper function from the library
@everywhere (ic_coupling_problem, ic_par, N_mc) = setup_ic_par_mc_problem(rp, ic_ranges, pars, (:K, K_range))

# for this first example we are only interested in the order parameter of the kuramoto model
@everywhere output_func = (sol, i) -> (order_parameter(sol[:,end], N), false)

# define the MC Problem
ko_mcp = MonteCarloProblem(rp, prob_func=ic_coupling_problem, output_func=output_func)
tail_frac = 0.9 # 
ko_emcp = EqMCProblem(ko_mcp, N_mc, tail_frac) 
ko_sol = solve(ko_emcp)




# Example 2: Logistic Map

In [ ]:
@everywhere r = 2.5:0.02:4
@everywhere pars = logistic_parameters(r[1])
@everywhere ic_ranges = [0.1:0.005:0.9]
@everywhere dp = DiscreteProblem(logistic, ic_ranges[1][1], (0.,500.), pars)
@everywhere (ic_r_prob, ic_par, N_mc) = setup_ic_par_mc_problem(dp, ic_ranges, pars, (:r, r))

log_mcp = MonteCarloProblem(dp, prob_func=ic_r_prob, output_func=eval_ode_run)
tail_frac = 0.8
log_emcp = EqMCProblem(log_mcp, N_mc, tail_frac)
log_sol = solve(log_emcp)

if isdefined(:cluster)
    @save "mc_log.jld2" log_sol
end 

In [ ]:
@load "mc_log.jld2" log_sol

In [ ]:
log_sol

In [ ]:
D = distance_matrix(log_sol);
k = 4
fdist = k_dist(D,k);

In [ ]:
x = collect(1:log_sol.N_mc)
Plots.plot(x[1:end],fdist[1:end],ylims=[0,0.5])

In [ ]:
eps = 0.15

In [ ]:
db_res = dbscan(D,eps,k)
cluster_meas = cluster_measures(db_res)

# Example 3: Roessler Network

In [ ]:
@everywhere N = 15

@everywhere k = 6
@everywhere p = 0.2
@everywhere net = watts_strogatz(N, k, p)

#@everywhere A = [0 1 1 
#    1 0 1 
#    1 1 0 ]
#@everywhere net = Graph(A)

@everywhere L = laplacian_matrix(net)

@everywhere a = ones(N).*0.2
@everywhere b = ones(N).*0.2
@everywhere c = ones(N).*7.0

tail_frac = 0.8

# for reference get the synchronizable range of Ks
evals = eig(full(L))[1]
evals = sort!(evals[evals .> 1e-5])
lambda_min = evals[end]
lambda_max = evals[1]
K_sync = (0.1232/lambda_min, 4.663/lambda_max)

In [ ]:
#@everywhere K_range = [0.002, 0.004, 0.006, 0.008, 0.010, 0.03, 0.05, 0.1, 0.2, 0.5, 1, 2, 4]
#@everywhere K_range = [0.002, 0.004, 0.03, 0.5, 1, 4]
#@everywhere K_range = [0.002, 0.005, 0.01, 0.025, 0.05, 0.07, 0.5, 1, 2]
@everywhere K_range = [0.005, 0.007, 0.009, 0.011, 0.013, 0.016, 0.02, 0.03, 0.04, 0.9, 1.5, 2]
@everywhere ic_gen_xy = Uniform(-15.,15.)
@everywhere ic_gen_z = Uniform(-5.,20.)

@everywhere ic_gens = [()->rand(ic_gen_xy), ()-> rand(ic_gen_xy), ()->rand(ic_gen_z)]
@everywhere N_ic = 100


@everywhere rp = ODEProblem(roessler_network, zeros(3*N), (0.,100.), roessler_parameters(a,b,c,0.05,L,N))
@everywhere (ic_coupling_problem, ic_par, N_mc) = setup_ic_par_mc_problem(rp, ic_gens, N_ic, roessler_parameters(a,b,c,0.05,L,N),(:K,K_range))


In [ ]:
rn_mcp = MonteCarloProblem(rp, prob_func=ic_coupling_problem, output_func=eval_ode_run_repeat)
rn_emcp = EqMCProblem(rn_mcp, N_mc, tail_frac)
@time rn_sol = solve(rn_emcp)

if isdefined(:cluster)
    @save "mc_roes_sol.jld2" rn_sol
    @save "mc_roes_ic_par.jld2" ic_par
end 

In [ ]:
@load "mc_roes_sol.jld2" rn_sol
@load "mc_roes_ic_par.jld2" ic_par

In [ ]:
D = distance_matrix(rn_sol);
k = 4
fdist = k_dist(D,k);

In [ ]:
x = collect(1:rn_sol.N_mc)
Plots.plot(x[1:end],fdist[1:end])

In [ ]:
eps = 6e8
db_res = dbscan(D,eps,k)
cluster_meas = cluster_measures(rn_sol, db_res)

# Example 4: Lotka Volterra

High Dimensional (N>=4) Lotka Volterra system can exhibit all kinds of interessting dynamics. Maybe it makes sense to study them with these tools

First we study the 4-dim model: $\frac{dN_i}{dt} = b_iN_i\left(1-\sum_{j=1}^n a_{ij}N_j\right),\qquad 1\leq i\leq n$

The parameter configuration

$ a = \left[ \begin{array}{cccc}
        1 & 1.09 & 1.52 & 0\\
        0 & 1 & 0.44 & 1.36\\
        2.33 & 0 & 1 & 0.47\\
        1.21 & 0.51 & 0.35 & 1\\
        \end{array} \right] $
        
$ b = \left(1, 0.72, 1.53, 1.27\right)$

is known to exhibit chaos with a strange attractor. Lets scan around these parameters.

In [1]:
@everywhere N = 4

@everywhere a = [1 1.09 1.52 0
                 0 1 0.44 1.36
                 2.33 0 1 0.47
                 1.21 0.51 0.35 1
                ]
@everywhere b = [1, 0.72, 1.53, 1.27]


In [2]:
a

4×4 Array{Float64,2}:
 1.0   1.09  1.52  0.0 
 0.0   1.0   0.44  1.36
 2.33  0.0   1.0   0.47
 1.21  0.51  0.35  1.0 

# Example 5: Henon Map